<a href="https://colab.research.google.com/github/ramesh1703/AlgoCasts/blob/master/Tic_Tac_Toe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip -q install anthropic google-generativeai gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.8/292.8 kB 6.4 MB/s eta 0:00:00


In [ ]:
import gradio as gr
import random
from openai import OpenAI
from google.colab import userdata
import os
from anthropic import Anthropic, HUMAN_PROMPT, AI_PROMPT
import google.generativeai as genai

In [ ]:
LLM_FRONTIERS = ["GPT-Frontier", "Claude-Frontier", "Gemini-Frontier", "Mistral-Frontier"]

In [ ]:
def model_map(model_name):
    """Maps a friendly model name to its API identifier."""
    return {
        "GPT-Frontier": "gpt-4o",
        "Claude-Frontier": "claude-3-5-sonnet-20241022",
        "Gemini-Frontier": "gemini-1.5-flash",
    }.get(model_name, "gpt-4o")

In [ ]:
openai_token = userdata.get('OPENAI_API_KEY')
anthropic_token = userdata.get('ANTHROPIC_API_KEY')
gemini_token = userdata.get('GEMINI_API_KEY')

openai_client = OpenAI(api_key=openai_token)
anthropic_client = Anthropic(api_key=anthropic_token)
genai.configure(api_key=gemini_token)

In [ ]:
def empty_board():
    """Returns an empty 3x3 Tic-Tac-Toe board."""
    return [["" for _ in range(3)] for _ in range(3)]

In [ ]:
def check_winner(board):
    """Checks for a winner or a draw on the board."""
    lines = (
        board +
        list(zip(*board)) +
        [[board[i][i] for i in range(3)]] +
        [[board[i][2 - i] for i in range(3)]]
    )

    for line in lines:
        if line[0] != '' and line.count(line[0]) == 3:
            return line[0]

    if all(cell != "" for row in board for cell in row):
        return "Draw"

    return None

In [ ]:
def render_board_markdown(board):
    """Converts the board list into a Markdown table string."""
    markdown_str = "| 0 | 1 | 2 |\n|---|---|---|\n"
    for r_idx, row in enumerate(board):
        row_str = "|"
        for c_idx, cell in enumerate(row):
            display_char = cell if cell != "" else "&nbsp;&nbsp;"
            row_str += f" {display_char} |"
        markdown_str += row_str + "\n"
    return markdown_str

In [ ]:
def llm_move(board, symbol, model_name="gpt-4o"):
    """
    Gets the LLM's move and updates the board.
    Returns the updated board and the raw LLM response text.
    """
    board_str = "\n".join(
        " ".join(cell or "." for cell in row)
        for row in board
    )

    prompt = (
        f"You are playing Tic-Tac-Toe as {symbol}.\n"
        f"Here is the current board:\n{board_str}\n"
        f"Reply with your move as two numbers 0-2 (row and column) separated by a space, "
        f"like '1 2'. Do not say anything else."
    )

    move_text = None
    try:
        if model_name.startswith("gpt-"):
            response = openai_client.chat.completions.create(
                model=model_name,
                messages=[{"role": "user", "content": prompt}],
                max_tokens=20,
                temperature=0.7
            )
            move_text = response.choices[0].message.content.strip()
        elif model_name.startswith("claude-"):
            response = anthropic_client.messages.create(
                model=model_name,
                max_tokens=20,
                messages=[{"role": "user", "content": prompt}]
            )
            move_text = response.content[0].text.strip()
        elif model_name.startswith("gemini-"):
            model = genai.GenerativeModel(model_name)
            response = model.generate_content(prompt)
            move_text = response.text.strip()
        else:

            print(f"[WARN] Unknown model: {model_name} - defaulting to random move")
            move_text = None
    except Exception as e:
        print(f"[ERROR] API call failed for {model_name}: {e}")
        move_text = f"ERROR: {e}"


    try:
        if move_text:
            i, j = map(int, move_text.split())
            if 0 <= i <= 2 and 0 <= j <= 2 and board[i][j] == "":
                board[i][j] = symbol
                return board, move_text
    except (ValueError, IndexError):
        pass

    empty = [(x, y) for x in range(3) for y in range(3) if board[x][y] == ""]
    if empty:
        i, j = random.choice(empty)
        board[i][j] = symbol
        fallback_msg = f"LLM invalid move ('{move_text}'). Placed '{symbol}' at fallback {i} {j}."
        return board, fallback_msg

    return board, move_text

In [ ]:
def play_turn(player1_model, player2_model, board_state, turn):
    """Handles one turn of the game."""

    if not player1_model or not player2_model:
        return board_state, "Please select both players", turn, render_board_markdown(board_state), "N/A"

    winner = check_winner(board_state)
    if winner:
        return board_state, f"Game Over! Winner: {winner}", turn, render_board_markdown(board_state), "Game Over"

    current_symbol = "X" if turn % 2 == 0 else "O"
    current_model_name = player1_model if current_symbol == "X" else player2_model
    mapped_model = model_map(current_model_name)

    llm_raw_output = "Thinking..."
    try:
        updated_board_state, llm_raw_output = llm_move(board_state, current_symbol, model_name=mapped_model)
    except Exception as e:
        return board_state, f"Error during LLM move: {str(e)}", turn, render_board_markdown(board_state), f"Error: {e}"

    winner = check_winner(updated_board_state)
    if winner:
        return updated_board_state, f"Game Over! Winner: {winner}", turn, render_board_markdown(updated_board_state), llm_raw_output
    else:
        return updated_board_state, f"Turn {turn + 1}: {current_model_name} played {current_symbol}", turn + 1, render_board_markdown(updated_board_state), llm_raw_output


In [ ]:
def reset_game():
    """Resets the game to its initial state."""
    new_board = empty_board()
    return new_board, "Press 'Next Turn' to start", 0, render_board_markdown(new_board), "N/A"

In [ ]:
def get_available_models(selected_model):
    """Filters available models based on the other selected model."""
    models = [m for m in LLM_FRONTIERS if m != selected_model]
    return models if models else LLM_FRONTIERS

In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("## 🤖 Tic Tac Toe: Two LLM Frontiers Play")

    with gr.Row():
        with gr.Column():
            player1_model = gr.Dropdown(choices=LLM_FRONTIERS, label="Player 1 Model (X)")
        with gr.Column():
            player2_model = gr.Dropdown(choices=LLM_FRONTIERS, label="Player 2 Model (O)")

    player1_model.change(lambda sel: gr.update(choices=get_available_models(sel)), inputs=player1_model, outputs=player2_model)
    player2_model.change(lambda sel: gr.update(choices=get_available_models(sel)), inputs=player2_model, outputs=player1_model)

    board_state = gr.State(value=empty_board())
    turn = gr.State(value=0)

    status = gr.Textbox(label="Game Status", value="Press 'Next Turn' to start")
    board_markdown = gr.Markdown(value=render_board_markdown(empty_board()), label="Board")
    llm_raw_output_display = gr.Textbox(label="Last LLM Raw Output", value="N/A", lines=3, interactive=False)

    with gr.Row():
        next_btn = gr.Button("Next Turn")
        reset_btn = gr.Button("Reset Game")

    next_btn.click(
        fn=play_turn,
        inputs=[player1_model, player2_model, board_state, turn],
        outputs=[board_state, status, turn, board_markdown, llm_raw_output_display]
    )

    reset_btn.click(
        fn=reset_game,
        inputs=[],
        outputs=[board_state, status, turn, board_markdown, llm_raw_output_display]
    )


In [ ]:
demo.launch(inbrowser=True)